# Hackaton EARIA 2019

Groupe _KaaRIs_
> **Agnès** (LIP6) : Chef d'équipe, algorithmes de ranking et de diversité des résultats  
> **Léo** (LIMSI) : Extraction et nettoyage des documents web  
> **Aymen** : Recherches sur le résumé de texte par sélection  
> **André** (Adesoft) : Fonctions utilitaires et rédaction des commentaires  

Les fausses nouvelles sont de plus en plus fréquentes dans l'actualité, et avec elles les travaux de vérification et de démentis de ces nouvelles. Certains journaux ont à cet effet créé des rubriques dédiées à ces travaux, comme la section des Décodeurs créée par les journaliste du Monde, ou les Fact Checkers de Libération. C'est sur ces derniers que porte le sujet de ce hackathon.

L'objectif de ce projet est d'établir des pistes pour répondre à la problématique du fact-checking des fake news. Plus formellement, il s'agira de proposer des articles de réponse, accompagnés de sources pertinentes, à des questions posées par des internautes au sujet d'informations suspectes non vérifiées. Pour cela, on dispose d'un corpus constitué d'environ 3000 questions posées aux journalistes de Libération, accompagnées pour la moitié d'entre elles des réponses apportées et des documents cités comme sources.

Il s'agit pour chaque question de rendre un document textuel comparable à la réponse des journalistes et au contenu des documents cités pour appuyer cette réponse.

On bénéficie pour cette session de hackathon d'un accès privilégié à l'API de recherche du moteur de recherche Qwant. Les réponses de cette API seront évidemment filtrées pour éliminer les articles de réponse des fact-checker de Libération, pour éviter de biaiser les résultats.

### Imports

In [610]:
import re
import nltk
from nltk.util import ngrams
import pandas as pd
from os import listdir
import string
import urllib.request as urlr
import urllib.parse as urlp
from urllib.request import Request
import json
from nltk.corpus import wordnet
from collections import Counter
import datetime
import requests
from bs4 import BeautifulSoup
from bs4.element import Comment
import pandas as pd
import os
import time
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import KMeans
from sklearn.metrics import adjusted_rand_score

### API Qwant

Basé sur du code fourni pour l'hackathon.

In [1]:
BASEURL="http://192.168.1.1:8000/qwant"
USER="kaaRIs"
PASSWORD="***************"

In [663]:
def qwant_raw(q):
    query="user={}&pass={}&q={}".format(urlp.quote(USER),urlp.quote(PASSWORD),urlp.quote(q))
    #print("{}/query?{}".format(BASEURL,query))
    r = urlr.urlopen("{}/query?{}".format(BASEURL,query))
    x = r.read().decode("utf-8")
    response = dict()
    if len(x):
        response = json.loads(x)
    return response

def qwant(q):
    response = qwant_raw(q)
    links = [r['url'] for r in response.get('result', dict()).get('items', [])]
    now = datetime.datetime.now()
    print("[", now.hour, now.minute, now.second, "]   Requête traitée :", q[:40], "... -", len(links), "liens trouvés.")
    return links

def get_doc(url):
    query="user={}&pass={}&url={}".format(urlp.quote(USER),urlp.quote(PASSWORD),urlp.quote(url))
    r = urlr.urlopen("{}/document?{}".format(BASEURL,query))
    
    return r.read().decode("utf-8", errors="ignore")

### Data

Chargement des données fournies : ensemble d'entraînement (questions posées, réponses apportées, et liste des documents fournis en source), ensemble de test (questions seules), et contenu des documents utliss dans l'ensemble d'entraînement.

In [ ]:
TRAIN_DATASET = pd.read_json("trainQD.json").set_index('id')
TEST_DATASET  = pd.read_json("testQD.json").set_index('id')
FILES_DATASET = pd.DataFrame([[int(document.split(".")[0]), open("train/" + document).read()] for document in listdir("train")]).set_index(0)

In [121]:
TEST_DATASET.sample(3)

,question
id,
2585,Est-ce qu'un «black face» est répréhensible pa...
750,Peter Madsen a été condamné à la prison à vie ...
140,Est-ce que seuls les bus de banlieue ont été a...


In [122]:
TRAIN_DATASET.sample(3)

,answer,docs,question
id,,,
1269,Question posée par Ber33 le 01/06/2018\nBonjou...,[7231],Qui est le vrai voisin de l'enfant du balcon?
2495,"Question posée par le 16/01/2019\nGuten Tag,\n...","[1924, 796, 6532, 3303, 10632, 3463, 2951, 522...",Que prévoit vraiment le traité d'Aix-la-Chapel...
280,"Question posée par le 22/11/2018\nBonjour,\nNo...","[9515, 7231]",Doit-on attendre janvier 2019 pour bénéficier ...


In [123]:
FILES_DATASET.sample(3)

,1
0,
9968,__\n\n__\n\n * [Home](https://www.tnp.no/)\n ...
11849,Lettre du garde des Sceaux à un futur ministre...
10386,\n\n


### Utils

#### Word processing

In [686]:
BLACKLIST = {"aujourd'hui", "quelqu'un", "affirme", "bien", "oui", "site", "lien", "vidéo", "article", "contre", "pour", "non", "plus", "dit", "questions", "monde", "libé", "facebook", "twitter", "vrai", "merci", "libération", "combien", "signaler", "signalé", "vraiment", "est", "ce", "que", "journal", "bonjour", "vérifier", "vérifiez"}
STOPWORDS = BLACKLIST | {'a', 'abord', 'absolument', 'afin', 'ah', 'ai', 'aie', 'aient', 'aies', 'ailleurs', 'ainsi', 'ait', 'allaient', 'allo', 'allons', 'allô', 'alors', 'anterieur', 'anterieure', 'anterieures', 'apres', 'après', 'as', 'assez', 'attendu', 'au', 'aucun', 'aucune', 'aucuns', 'aujourd', "aujourd'hui", 'aupres', 'auquel', 'aura', 'aurai', 'auraient', 'aurais', 'aurait', 'auras', 'aurez', 'auriez', 'aurions', 'aurons', 'auront', 'aussi', 'autre', 'autrefois', 'autrement', 'autres', 'autrui', 'aux', 'auxquelles', 'auxquels', 'avaient', 'avais', 'avait', 'avant', 'avec', 'avez', 'aviez', 'avions', 'avoir', 'avons', 'ayant', 'ayez', 'ayons', 'b', 'bah', 'bas', 'basee', 'bat', 'beau', 'beaucoup', 'bien', 'bigre', 'bon', 'boum', 'bravo', 'brrr', 'c', 'car', 'ce', 'ceci', 'cela', 'celle', 'celle-ci', 'celle-là', 'celles', 'celles-ci', 'celles-là', 'celui', 'celui-ci', 'celui-là', 'celà', 'cent', 'cependant', 'certain', 'certaine', 'certaines', 'certains', 'certes', 'ces', 'cet', 'cette', 'ceux', 'ceux-ci', 'ceux-là', 'chacun', 'chacune', 'chaque', 'cher', 'chers', 'chez', 'chiche', 'chut', 'chère', 'chères', 'ci', 'cinq', 'cinquantaine', 'cinquante', 'cinquantième', 'cinquième', 'clac', 'clic', 'combien', 'comme', 'comment', 'comparable', 'comparables', 'compris', 'concernant', 'contre', 'couic', 'crac', 'd', 'da', 'dans', 'de', 'debout', 'dedans', 'dehors', 'deja', 'delà', 'depuis', 'dernier', 'derniere', 'derriere', 'derrière', 'des', 'desormais', 'desquelles', 'desquels', 'dessous', 'dessus', 'deux', 'deuxième', 'deuxièmement', 'devant', 'devers', 'devra', 'devrait', 'different', 'differentes', 'differents', 'différent', 'différente', 'différentes', 'différents', 'dire', 'directe', 'directement', 'dit', 'dite', 'dits', 'divers', 'diverse', 'diverses', 'dix', 'dix-huit', 'dix-neuf', 'dix-sept', 'dixième', 'doit', 'doivent', 'donc', 'dont', 'dos', 'douze', 'douzième', 'dring', 'droite', 'du', 'duquel', 'durant', 'dès', 'début', 'désormais', 'e', 'effet', 'egale', 'egalement', 'egales', 'eh', 'elle', 'elle-même', 'elles', 'elles-mêmes', 'en', 'encore', 'enfin', 'entre', 'envers', 'environ', 'es', 'essai', 'est', 'et', 'etant', 'etc', 'etre', 'eu', 'eue', 'eues', 'euh', 'eurent', 'eus', 'eusse', 'eussent', 'eusses', 'eussiez', 'eussions', 'eut', 'eux', 'eux-mêmes', 'exactement', 'excepté', 'extenso', 'exterieur', 'eûmes', 'eût', 'eûtes', 'f', 'fais', 'faisaient', 'faisant', 'fait', 'faites', 'façon', 'feront', 'fi', 'flac', 'floc', 'fois', 'font', 'force', 'furent', 'fus', 'fusse', 'fussent', 'fusses', 'fussiez', 'fussions', 'fut', 'fûmes', 'fût', 'fûtes', 'g', 'gens', 'h', 'ha', 'haut', 'hein', 'hem', 'hep', 'hi', 'ho', 'holà', 'hop', 'hormis', 'hors', 'hou', 'houp', 'hue', 'hui', 'huit', 'huitième', 'hum', 'hurrah', 'hé', 'hélas', 'i', 'ici', 'il', 'ils', 'importe', 'j', 'je', 'jusqu', 'jusque', 'juste', 'k', 'l', 'la', 'laisser', 'laquelle', 'las', 'le', 'lequel', 'les', 'lesquelles', 'lesquels', 'leur', 'leurs', 'longtemps', 'lors', 'lorsque', 'lui', 'lui-meme', 'lui-même', 'là', 'lès', 'm', 'ma', 'maint', 'maintenant', 'mais', 'malgre', 'malgré', 'maximale', 'me', 'meme', 'memes', 'merci', 'mes', 'mien', 'mienne', 'miennes', 'miens', 'mille', 'mince', 'mine', 'minimale', 'moi', 'moi-meme', 'moi-même', 'moindres', 'moins', 'mon', 'mot', 'moyennant', 'multiple', 'multiples', 'même', 'mêmes', 'n', 'na', 'naturel', 'naturelle', 'naturelles', 'ne', 'neanmoins', 'necessaire', 'necessairement', 'neuf', 'neuvième', 'ni', 'nombreuses', 'nombreux', 'nommés', 'non', 'nos', 'notamment', 'notre', 'nous', 'nous-mêmes', 'nouveau', 'nouveaux', 'nul', 'néanmoins', 'nôtre', 'nôtres', 'o', 'oh', 'ohé', 'ollé', 'olé', 'on', 'ont', 'onze', 'onzième', 'ore', 'ou', 'ouf', 'ouias', 'oust', 'ouste', 'outre', 'ouvert', 'ouverte', 'ouverts', 'o|', 'où', 'p', 'paf', 'pan', 'par', 'parce', 'parfois', 'parle', 'parlent', 'parler', 'parmi', 'parole', 'parseme', 'partant', 'particulier', 'particulière', 'particulièrement', 'pas', 'passé', 'pendant', 'pense', 'permet', 'personne', 'personnes', 'peu', 'peut', 'peuvent', 'peux', 'pff', 'pfft', 'pfut', 'pif', 'pire', 'pièce', 'plein', 'plouf', 'plupart', 'plus', 'plusieurs', 'plutôt', 'possessif', 'possessifs', 'possible', 'possibles', 'pouah', 'pour', 'pourquoi', 'pourrais', 'pourrait', 'pouvait', 'prealable', 'precisement', 'premier', 'première', 'premièrement', 'pres', 'probable', 'probante', 'procedant', 'proche', 'près', 'psitt', 'pu', 'puis', 'puisque', 'pur', 'pure', 'q', 'qu', 'quand', 'quant', 'quant-à-soi', 'quanta', 'quarante', 'quatorze', 'quatre', 'quatre-vingt', 'quatrième', 'quatrièmement', 'que', 'quel', 'quelconque', 'quelle', 'quelles', "quelqu'un", 'quelque', 'quelques', 'quels', 'qui', 'quiconque', 'quinze', 'quoi', 'quoique', 'r', 'rare', 'rarement', 'rares', 'relative', 'relativement', 'remarquable', 'rend', 'rendre', 'restant', 'reste', 'restent', 'restrictif', 'retour', 'revoici', 'revoilà', 'rien', 's', 'sa', 'sacrebleu', 'sait', 'sans', 'sapristi', 'sauf', 'se', 'sein', 'seize', 'selon', 'semblable', 'semblaient', 'semble', 'semblent', 'sent', 'sept', 'septième', 'sera', 'serai', 'seraient', 'serais', 'serait', 'seras', 'serez', 'seriez', 'serions', 'serons', 'seront', 'ses', 'seul', 'seule', 'seulement', 'si', 'sien', 'sienne', 'siennes', 'siens', 'sinon', 'six', 'sixième', 'soi', 'soi-même', 'soient', 'sois', 'soit', 'soixante', 'sommes', 'son', 'sont', 'sous', 'souvent', 'soyez', 'soyons', 'specifique', 'specifiques', 'speculatif', 'stop', 'strictement', 'subtiles', 'suffisant', 'suffisante', 'suffit', 'suis', 'suit', 'suivant', 'suivante', 'suivantes', 'suivants', 'suivre', 'sujet', 'superpose', 'sur', 'surtout', 't', 'ta', 'tac', 'tandis', 'tant', 'tardive', 'te', 'tel', 'telle', 'tellement', 'telles', 'tels', 'tenant', 'tend', 'tenir', 'tente', 'tes', 'tic', 'tien', 'tienne', 'tiennes', 'tiens', 'toc', 'toi', 'toi-même', 'ton', 'touchant', 'toujours', 'tous', 'tout', 'toute', 'toutefois', 'toutes', 'treize', 'trente', 'tres', 'trois', 'troisième', 'troisièmement', 'trop', 'très', 'tsoin', 'tsouin', 'tu', 'té', 'u', 'un', 'une', 'unes', 'uniformement', 'unique', 'uniques', 'uns', 'v', 'va', 'vais', 'valeur', 'vas', 'vers', 'via', 'vif', 'vifs', 'vingt', 'vivat', 'vive', 'vives', 'vlan', 'voici', 'voie', 'voient', 'voilà', 'vont', 'vos', 'votre', 'vous', 'vous-mêmes', 'vu', 'vé', 'vôtre', 'vôtres', 'w', 'x', 'y', 'z', 'zut', 'à', 'â', 'ça', 'ès', 'étaient', 'étais', 'était', 'étant', 'état', 'étiez', 'étions', 'été', 'étée', 'étées', 'étés', 'êtes', 'être', 'ô', "a", "ai", "aie", "aient", "aies", "ait", "alors", "as", "au", "aucun", "aura", "aurai", "auraient", "aurais", "aurait", "auras", "aurez", "auriez", "aurions", "aurons", "auront", "aussi", "autre", "aux", "avaient", "avais", "avait", "avant", "avec", "avez", "aviez", "avions", "avoir", "avons", "ayant", "ayez", "ayons", "bon", "car", "ce", "ceci", "cela", "ces", "cet", "cette", "ceux", "chaque", "ci", "comme", "comment", "d", "dans", "de", "dedans", "dehors", "depuis", "des", "deux", "devoir", "devrait", "devrez", "devriez", "devrions", "devrons", "devront", "dois", "doit", "donc", "dos", "droite", "du", "dès", "début", "dù", "elle", "elles", "en", "encore", "es", "est", "et", "eu", "eue", "eues", "eurent", "eus", "eusse", "eussent", "eusses", "eussiez", "eussions", "eut", "eux", "eûmes", "eût", "eûtes", "faire", "fais", "faisez", "fait", "faites", "fois", "font", "force", "furent", "fus", "fusse", "fussent", "fusses", "fussiez", "fussions", "fut", "fûmes", "fût", "fûtes", "haut", "hors", "ici", "il", "ils", "j", "je", "juste", "l", "la", "le", "les", "leur", "leurs", "lui", "là", "m", "ma", "maintenant", "mais", "me", "mes", "moi", "moins", "mon", "mot", "même", "n", "ne", "ni", "nom", "nommé", "nommée", "nommés", "nos", "notre", "nous", "nouveau", "nouveaux", "on", "ont", "ou", "où", "par", "parce", "parole", "pas", "personne", "personnes", "peu", "peut", "plupart", "pour", "pourquoi", "qu", "quand", "que", "quel", "quelle", "quelles", "quels", "qui", "sa", "sans", "se", "sera", "serai", "seraient", "serais", "serait", "seras", "serez", "seriez", "serions", "serons", "seront", "ses", "seulement", "si", "sien", "soi", "soient", "sois", "soit", "sommes", "son", "sont", "sous", "soyez", "soyons", "suis", "sujet", "sur", "t", "ta", "tandis", "te", "tellement", "tels", "tes", "toi", "ton", "tous", "tout", "trop", "très", "tu", "un", "une", "valeur", "voient", "vois", "voit", "vont", "vos", "votre", "vous", "vu", "y", "à", "ça", "étaient", "étais", "était", "étant", "état", "étiez", "étions", "été", "étés", "êtes", "être"}

def tokenize(text):
    return nltk.word_tokenize(re.sub(r'\W+', ' ', text.lower()), language='french')

def clean(tokens):
    # Stop words
    data = [word for word in tokens if word not in STOPWORDS]
    
    # Symbols
    table = str.maketrans('', '', string.punctuation + "`’0123456789")
    data = [w.translate(table) for w in data]
    
    # Useless words
    data = [w for w in data if len(w) < 15 and len(w) > 1]
    
    # Whitespaces
    data = [w for w in data if w != '']
    
    return data

def add_syns(tokens, syns_number):
    # Adds up to syns_number synonyms for each word in tokens.
    # On the hypothesis that more synonyms may help to increase qwant results precision.
    syns = set([w for word in tokens for t in wordnet.synsets(word, lang='fra') for w in t.lemma_names('fra')[:syns_number]])
    tokens += syns
    return tokens

def ngramize(tokens, n):
    if n < 2:
        return tokens
    return ngrams(tokens, n)

def count(ngrams):
    return Counter(ngrams)

### Traitement

Une étape intermédiaire utile consiste ici à effectuer une petite analyse du corpus de test. On a déjà l'intuition que des informations tels que des mots gramaticaux, ou des symboles de poncuation, n'apporteraient rien dans une requête soumise à un moteur de recherche. Nous débarsserons donc les questions de l'ensemble de test avant de les soumettre à Qwant.

In [688]:
def process_query(text, s=1):
    data = text
    data = tokenize(data)
    data = clean(data)
    data = add_syns(data, s)
    return ' '.join(data)

def process_doc(text, n=3):
    data = text
    data = tokenize(data)
    data = clean(data)
    #data = ngramize(data, n)
    return ' '.join(data)

### Analyse

On étudie succintement les données disponibles pour la phase de test. Pour cela, on effectue un compte de tous les mots constituant les requêtes à soumettre (questions après traitement de nettoyage). On remarque alors que certains mots reviennent très souvent, alors qu'il n'est aucunnement pertinent de les utiliser dans une recherche, comme les mots interrogatifs, des adverbes abstraits comme "aujourd'hui" ou "quelqu'un", des verbes généralistes comme "dit" ou "affirme", ou encore des noms faisant référence à des des types de ressources, comme "article" ou "vidéo".

Il est compliqué d'effectuer une reconnaissance d'entitées nommées ou un traitement similaire, mais on peut gagner significativement en efficacité en éliminant ces termes inutiles.

On les ajoute à la main à la liste des termes à supprimer (stopwords). On ne peut pas simplement éliminer les mots les plus courants, car le corpus est fondamentalement biaisé : des mots comme "Macron", "France", ou "gilets" et "jaunes" apparaissent un grand nombre de fois pour des raisons légitimes : il s'agit de sujet faisant l'objet de nombreuses actualités, et donc mécaniquement de nombreuses informations à vérifier.

In [559]:
Counter(process_query(' '.join(list(TEST_DATASET.question))).split(" ")).most_common(30)

[('macron', 111),
 ('france', 110),
 ('va', 48),
 ('français', 43),
 ('euros', 36),
 ('gilets', 35),
 ('jaunes', 34),
 ('lors', 34),
 ('entre', 30),
 ('mélenchon', 29),
 ('paris', 27),
 ('emmanuel', 26),
 ('compte', 25),
 ('affaire', 22),
 ('après', 22),
 ('pendant', 22),
 ('trump', 22),
 ('gouvernement', 21),
 ('etat', 21),
 ('pays', 21),
 ('migrants', 20),
 ('jean', 20),
 ('homme', 19),
 ('police', 19),
 ('hausse', 18),
 ('campagne', 18),
 ('députés', 18),
 ('policiers', 18),
 ('dire', 17),
 ('concernant', 17)]

### Requêtes

In [489]:
TEST_DATASET['request'] = None
TEST_DATASET['request'] = [process_query(t[1].question, 0) for t in TEST_DATASET.iterrows()]

In [309]:
TRAIN_DATASET['request'] = None
TRAIN_DATASET['request'] = [process_query(t[1].question, 0) for t in TRAIN_DATASET.iterrows()]

### Appels API

In [583]:
for index, row in TEST_DATASET.iterrows():
    # If request has already succeded, dont do it again
    # None indicates a request has never been treated.
    # Empty list commonly indicates a temporary fail in qwant research process.
    if row['links'] is None or len(row['links']) == 0:
        row['links'] = qwant(row.request)

[ 18 37 5 ]   Requête traitée : frais scolarité coûteront chers étudiant ... - 48 liens trouvés.


KeyboardInterrupt: 

### Traitement pages HTML

Grâce au code fourni pour le hackathon, on peut effectuer une requête de recherche sur Qwant et récupérer le contenu des premiers résultats correspondants. Afin d'extraire le contenu intéressant de ces documents, on effectue un traitement de la page HTML.

In [658]:
def scrap(html):
    limite = 100
 
    def tag_visible(elementT):
        # We delete everything that is irrevelant to the page content.
        # FIXME: Maybe should we keep the title tag ?
        for element in elementT.parents:
            if element.name in ['style', 'script', 'head', 'title', 'meta', 'footer', 'header']:
                return False
        if isinstance(elementT, Comment):
            return False
        return True
 
    def parse_article(soup):
       
        # Main article commonly starts after the (first) first-levle title
        h1 = soup.find('h1')
        fullText = ""
 
        # Main content is commonly into <p> tags
        root = soup.body
        if not root.find('p'):
            return ""
        ps = root.find_all('p', recursive=True)
 
        # The main content is often the most massive text bloc in the page.
        # So we can look for where are the most <p> tags, to find the most important text
        # Without any sentence-like texte recognition.
        currMax = 0
        currPmax = None
        for p in ps:
            #if p.has_key('class'):
                #print(p.parent["class"])
            #print(len(p.parent.find_all('p', recursive=False)))
            if len(p.parent.find_all('p', recursive=False)) > currMax:
                currMax = len(p.parent.find_all('p', recursive=False))
                currPmax = p.parent
 
        if currMax == 0:
            print('nope')
            return ""
        tex = currPmax.find_all(text=True)
        visible_texts = filter(tag_visible, tex)  
        fullText = u" ".join(t.strip() for t in visible_texts)
 
        return h1.text + "\n" + fullText
 
    def get_article(url):
        res = requests.get(url)
        return parse_article(res.content)
   
    soup = BeautifulSoup(html, 'html.parser')
    res = parse_article(soup)
    taille = len(res.split())
 
    if taille < limite:
        tex = soup.findAll(text=True)
 
        visible_texts = filter(tag_visible, tex)  
        textWrite = u" ".join(t.strip() for t in visible_texts)
        return textWrite
    else:
        return res

#r = requests.get('https://fr.news.yahoo.com/brigitte-macron-pourquoi-pris-distances-143024731.html')
 
#res = scrap(r.text)
#print(res)
#taille = len(res.split())
#print(taille)

### Diversité

Pour la sélection des passages les plus pertinents constitutifs de l'article de réponse à rendre (ou résumé), on se propose d'effectuer un travail de diversité sur les documents. L'idée est d'effectuer un rapide clustering sur les documents recupérés suite à la requête Qwant, et sélectionner le plus pertinent de chaque cluster. Les documents choisis serviront à constituer le résumé rendu.

L'objectif est de maximiser la couverture d'information par le résumé tout en limitant sa longueur.

In [614]:
class Data(object):
    def __init__(self, directory):
        self.corpus, self.idx_2_name = self.create_corpus(directory)

    # Haha, we used the same method name in the same time. This is oddly funny. Now process my burial please x)
    def process_doc(self, doc_path):
        pattern = re.compile('[\W_]+')
        with open(doc_path, 'r') as f:
            d = f.readlines()
            d = ' '.join(d)
            d = pattern.sub(' ', d).strip().lower()#.split()
        return d

    def create_corpus(self, directory):
        corpus = []
        idx_2_name = {}

        docs_filenames = os.listdir(directory)[:10]
        docs_path = [os.path.join(directory, f) for f in docs_filenames]

        for i, doc in enumerate(docs_filenames):
            doc_name = doc.split('.txt')[0]
            doc_path = os.path.join(directory, doc)
            idx_2_name[i] = int(doc_name)
            corpus.append(self.process_doc(doc_path))

        return corpus, idx_2_name



vectorizer = TfidfVectorizer(stop_words=STOPWORDS)
X = vectorizer.fit_transform(documents)

true_k = 3
model = KMeans(n_clusters=true_k, init='k-means++', max_iter=100, n_init=1)
model.fit(X)
clusters = list(model.predict(X))


pertinent_docs = []

for i in range(true_k):
    doc_idx = clusters.index(i)
    pertinent_docs.append(documents[doc_idx])

print(pertinent_docs)

NameError: name 'documents' is not defined

### Soumission

In [679]:
# Task 1 : submit for each question a list of revelant documents URL
def submit_docs(data):
    run = dict()
    for index, row in TEST_DATASET.iterrows():
        if row.links is None:
            run[index] = []
        else:
            run[index] = row.links[:10]

    query="user={}&pass={}".format(urlp.quote(USER), urlp.quote(PASSWORD))
    r = urlr.urlopen("http://192.168.1.1:8000/hackathon/submitrun?{}".format(query), data=json.dumps(run).encode("utf-8"))
    response = r.read()
    print(response)
    
# Task 2 : submit for each question one string that summurize the answer and the documents
def submit_text(data, summarize):
    run = dict()
    for index, row in TEST_DATASET.iterrows():
        run[index] = summarize(row)
    

    query="user={}&pass={}".format(urlp.quote(USER), urlp.quote(PASSWORD))
    r = urlr.urlopen("http://192.168.1.1:8000/hackathon/submitrun?{}".format(query), data=json.dumps(run).encode("utf-8"))
    response = r.read()
    print(response)

def submit_docs_shiny(data):
    run = dict()
    for index, row in data.iterrows():
        if row.links is None:
            run[index] = ""
        else:
            xd = []
            for doc in row.links[:2]:
                try:
                    xd.append(process_doc(scrap(get_doc(doc)))[:200])
                except Exception:
                    pass
            #[process_doc(scrap(get_doc(doc))) for doc in row.links[:2]]
            run[index] = " ".join(xd)+row.request
        print("Done")

    query="user={}&pass={}".format(urlp.quote(USER), urlp.quote(PASSWORD))
    r = urlr.urlopen("http://192.168.1.1:8000/hackathon/submitrun?{}".format(query), data=json.dumps(run).encode("utf-8"))
    response = r.read()
    print(response)
    
def get_start(data):
    ' '.join(nltk.tokenize.tokenize(data)[:10])
    
def summarize_brutal(data):
    return ' '.join([get_doc(doc) for doc in data[:3]])

def summarize_shiny(data):
    for doc in data:
        text = scrap(get_doc(doc))
        ' '.join(nltk.tokenize.tokenize(text)[:10])

In [680]:
submit_docs_shiny(TEST_DATASET.head(20))

Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
b'{"response": "queued"}'


#### Liste de documents

In [623]:
submit_docs(TEST_DATASET.iloc[0:20])

b'{"response": "queued"}'


#### Contenu des articles

In [683]:
def yolo(row):
    return row['request']

submit_text(TEST_DATASET, yolo)

b'{"response": "queued"}'


### Tests

In [689]:
# Processed questions
len([n for n in TEST_DATASET.links if n is not None and len(n) > 0])

716

In [690]:
# Failed questions
len([n for n in TEST_DATASET.links if n is not None and len(n) == 0])

4

In [691]:
# Unrocessed questions
len([n for n in TEST_DATASET.links if n is None])

638

In [692]:
# All questions
len(TEST_DATASET.links)

1358

In [392]:
TEST_DATASET.request[89]

'mamoudou gassama naturalisé régularisé'